In [ ]:
import time
import busio
from digitalio import DigitalInOut, Direction, Pull
import board
import adafruit_ssd1306
import adafruit_rfm9x
import subprocess  # For running external scripts
import signal  # For handling subprocess termination
import sys  # For restarting the script
import os

# Function to stop all subprocesses
def stop_subprocesses(subprocesses):
    for process in subprocesses:
        process.send_signal(signal.SIGINT)  # Send interrupt signal to terminate
        process.wait()

# Function to restart the script
def restart_script():
    python = sys.executable
    os.execl(python, python, *sys.argv)

# Button A
btnA = DigitalInOut(board.D5)
btnA.direction = Direction.INPUT
btnA.pull = Pull.UP

# Button B
btnB = DigitalInOut(board.D6)
btnB.direction = Direction.INPUT
btnB.pull = Pull.UP

# Button C
btnC = DigitalInOut(board.D12)
btnC.direction = Direction.INPUT
btnC.pull = Pull.UP

# Create the I2C interface.
i2c = busio.I2C(board.SCL, board.SDA)

# 128x32 OLED Display
reset_pin = DigitalInOut(board.D4)
display = adafruit_ssd1306.SSD1306_I2C(128, 32, i2c, reset=reset_pin)
width = display.width
height = display.height

# Configure LoRa Radio
CS = DigitalInOut(board.CE1)
RESET = DigitalInOut(board.D25)
spi = busio.SPI(board.SCK, MOSI=board.MOSI, MISO=board.MISO)
rfm9x = adafruit_rfm9x.RFM9x(spi, CS, RESET, 433)
rfm9x.tx_power = 23

subprocesses = []  # List to store subprocesses

while True:
    packet = None
    display.fill(0)
    display.text('Waiting Button Press\n1:Lock\n2:Unlock\n3:Register\nC:Stop', 0, 1, 1)
    display.show()

    if not btnA.value:
        display.fill(0)
        display.text('Running Auth Lock', 25, 15, 1)
        display.show()
        try:
            subprocesses.append(subprocess.Popen(["python3", "Authentication_lock.py"]))
        except Exception as e:
            display.fill(0)
            display.text(f'Error: {str(e)}', 10, 15, 1)
            display.show()
            time.sleep(5)
            restart_script()

    elif not btnB.value:
        display.fill(0)
        display.text('Running Auth Unlock', 10, 15, 1)
        display.show()
        try:
            subprocesses.append(subprocess.Popen(["python3", "Authentication_unlock.py"]))
        except Exception as e:
            display.fill(0)
            display.text(f'Error: {str(e)}',10, 15, 1)
            display.show()
            time.sleep(5)
            restart_script()

    elif not btnC.value:
        display.fill(0)
        display.text('Stopping Subprocesses', 10, 15, 1)
        display.show()
        stop_subprocesses(subprocesses)  # Stop all subprocesses
        subprocesses = []  # Clear the list

    time.sleep(1)
